<a href="https://colab.research.google.com/github/TMFGC/CNNM3/blob/main/Clasificaci%C3%B3n_Tranfer_learning_from_Classification_of_Leukemia_(Pytorch)_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install optuna

In [ ]:
!pip install Pillow

Importar librerias

In [ ]:
import os
import torch
import numpy as np
import random
from PIL import Image
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision
from torchvision import transforms, datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import optuna
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

print('Data source import complete.')


In [ ]:
# abrir archivo
data_cells= '/content/drive/My Drive/datasets/dataset_m3/train_dir'

In [ ]:
 os.getcwd()

In [ ]:
torch.cuda.is_available()

In [ ]:
print(f"PyTorch version: {torch.__version__}\ntorchvision version: {torchvision.__version__}")

In [ ]:
torch.cuda.device_count()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Cargar el dataset

In [ ]:
# Paths and configurations
data_cells = '/content/drive/My Drive/datasets/dataset_m3/train_dir'
BATCH_SIZE = 16
CLASSES = ['PMCA','SNE','MO','LY','BLAST']
IMAGE_SIZE = (144, 144)

In [ ]:
# Helper function to count files in directories
def count_files_in_directory(root_directory, target_folder):
    target_path = None
    for root, dirs, _ in os.walk(root_directory):
        if target_folder in dirs:
            target_path = os.path.join(root, target_folder)
            break
    if target_path:
        total_files = sum(len(files) for _, _, files in os.walk(target_path))
        print(f"Total number of files under the '{target_folder}' folder: {total_files}")
    else:
        print(f"No '{target_folder}' folder found.")

In [ ]:
for cls in CLASSES:
    count_files_in_directory(data_cells, cls)

CAMBIAR IMAGEN DE JPG A TIFF

In [ ]:
from PIL import Image
import os

def convert_jpg_to_tiff(data_cells, classes):
  for cls in classes:
    class_path = os.path.join(data_cells, cls)
    for filename in os.listdir(class_path):
      if filename.endswith(".jpg"):
        filepath = os.path.join(class_path, filename)
        with Image.open(filepath) as im:
          # Crea un nuevo nombre de archivo con extensión .tiff
          tiff_filepath = os.path.join(class_path, filename[:-4] + ".tiff")
          im.save(tiff_filepath, "TIFF")  # Guarda la imagen como TIFF
        # (Opcional) Elimina el archivo JPG original si ya no lo necesitas
        # os.remove(filepath)

# Llama a la función para convertir las imágenes
convert_jpg_to_tiff(data_cells, CLASSES)

In [ ]:
# Dataset class for loading images
class CustomDataset(Dataset):
    def __init__(self, data_cells, classes, image_batch_size=3000, transform=None):
        self.transform = transform
        self.images, self.labels = self.load_images(data_cells, classes, image_batch_size)

    def load_images(self, data_cells, classes, image_batch_size):
        images, labels = [], []
        for idx, class_name in enumerate(classes):
            class_path = os.path.join(data_cells, class_name)
            # Use os.listdir to get all files and filter by extension
            # The problem was here, the '.tif' files were actually '.tiff' files
            image_paths = [os.path.join(class_path, f) for f in os.listdir(class_path) if f.endswith('.tiff')]


            # Removed the image_batch_size limitation to load all images
            for img_path in image_paths:
                with Image.open(img_path) as img:
                    images.append(np.array(img.convert('RGB')))
                    labels.append(idx)
        return images, labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.transform(self.images[idx]) if self.transform else self.images[idx]
        return image, self.labels[idx]

In [ ]:
# Image transformations
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(IMAGE_SIZE),
    transforms.ToTensor()
])

Crear y dividir un conjunto de datos

In [ ]:
dataset = CustomDataset(data_cells=data_cells, classes=CLASSES, transform=transform)
train_size = int(0.8 * len(dataset))
val_size = int(0.15 * len(dataset))  # Calcular val_size basado en el dataset completo
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size]) # Dividir en 3 conjuntos

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

modelo

In [ ]:
class CNNModel(nn.Module):
    def __init__(self, num_layers, num_units, dropout_rate):
        super(CNNModel, self).__init__()
        layers = [nn.Conv2d(3, num_units, kernel_size=3, padding=1), nn.ReLU(),
                  nn.BatchNorm2d(num_units), nn.MaxPool2d(2)]

        for _ in range(num_layers - 1):
            layers.extend([nn.Conv2d(num_units, num_units * 2, kernel_size=3, padding=1),
                           nn.ReLU(), nn.BatchNorm2d(num_units * 2), nn.MaxPool2d(2)])
            num_units *= 2

        self.conv_layers = nn.Sequential(*layers)
        fc_input_size = num_units * (IMAGE_SIZE[0] // (2 ** num_layers)) * (IMAGE_SIZE[1] // (2 ** num_layers))
        self.fc1 = nn.Linear(fc_input_size, 512)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(512, 5)

    def forward(self, x):
        x = self.conv_layers(x).view(x.size(0), -1)
        return self.fc2(self.dropout(self.fc1(x)))

Hyperparameter Tunning

In [ ]:
EPOCHS = 15

In [ ]:
def objective(trial):
    num_layers = trial.suggest_int("num_layers", 2, 3)
    num_units = trial.suggest_int("num_units", 128, 384, step=64)
    dropout_rate = trial.suggest_float("dropout_rate", 0.0, 0.3)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-4, log=True)

    model = CNNModel(num_layers=num_layers, num_units=num_units, dropout_rate=dropout_rate).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    patience = 3
    best_val_loss = float('inf')
    epochs_without_improvement = 0
    for epoch in range(EPOCHS):
        model.train()
        train_loss = sum(criterion(model(inputs.to(device)), labels.to(device)).item() for inputs, labels in train_loader) / len(train_loader)
        val_loss = sum(criterion(model(inputs.to(device)), labels.to(device)).item() for inputs, labels in val_loader) / len(val_loader)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= patience:
                break
    return best_val_loss

In [ ]:
# Run Optuna optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=7)

[I 2025-05-04 21:41:23,430] A new study created in memory with name: no-name-395731fd-608c-4a3e-86da-67987cb66f73
[I 2025-05-04 21:53:41,424] Trial 0 finished with value: 1.589538186788559 and parameters: {'num_layers': 3, 'num_units': 192, 'dropout_rate': 0.14571073604503323, 'learning_rate': 6.024685934644941e-05}. Best is trial 0 with value: 1.589538186788559.
[I 2025-05-04 22:13:09,853] Trial 1 finished with value: 1.5798932909965515 and parameters: {'num_layers': 2, 'num_units': 384, 'dropout_rate': 0.028518823772527034, 'learning_rate': 2.7913958206714023e-05}. Best is trial 1 with value: 1.5798932909965515.
[I 2025-05-04 22:20:59,753] Trial 2 finished with value: 1.6125814020633698 and parameters: {'num_layers': 3, 'num_units': 128, 'dropout_rate': 0.22556623846390528, 'learning_rate': 3.928759344255263e-05}. Best is trial 1 with value: 1.5798932909965515.
[I 2025-05-04 22:29:07,267] Trial 3 finished with value: 1.4870198965072632 and parameters: {'num_layers': 3, 'num_units': 1

In [ ]:
best_params = study.best_params

print("  Hyperparameters: ")
for key, value in best_params.items():
    print(f"    {key}: {value}")

Training with Best Hyperparameters

In [ ]:
best_model = CNNModel(
    num_layers=best_params['num_layers'],
    num_units=best_params['num_units'],
    dropout_rate=best_params['dropout_rate']
).to(device)

In [ ]:
optimizer = optim.Adam(best_model.parameters(), lr=best_params['learning_rate'])
criterion = nn.CrossEntropyLoss()

In [ ]:
# Metrics from training
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

EPOCHS = 50
patience = 3
best_val_loss = float('inf')
epochs_without_improvement = 0

In [ ]:
for epoch in range(EPOCHS):
    # Training phase
    best_model.train()
    running_train_loss = 0.0
    correct_train = 0
    total_train = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = best_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_train_loss += loss.item()

        # Calculate training accuracy
        _, predicted = torch.max(outputs, 1)
        correct_train += (predicted == labels).sum().item()
        total_train += labels.size(0)

    avg_train_loss = running_train_loss / len(train_loader)
    train_accuracy = correct_train / total_train
    train_losses.append(avg_train_loss)
    train_accuracies.append(train_accuracy)

    # Validation phase
    best_model.eval()
    running_val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = best_model(inputs)
            loss = criterion(outputs, labels)
            running_val_loss += loss.item()

            # Calculate validation accuracy
            _, predicted = torch.max(outputs, 1)
            correct_val += (predicted == labels).sum().item()
            total_val += labels.size(0)

    avg_val_loss = running_val_loss / len(val_loader)
    val_accuracy = correct_val / total_val
    val_losses.append(avg_val_loss)
    val_accuracies.append(val_accuracy)

    print(f'Epoch {epoch+1}: Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.4f}, '
          f'Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.4f}')

    # Early stopping check
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1
        if epochs_without_improvement >= patience:
            print("Early stopping triggered")
            break

In [ ]:
# Plot training and validation losses and accuracies
def plot_metrics(train_losses, val_losses, train_accuracies, val_accuracies):
    epochs_range = range(1, len(train_losses) + 1)
    plt.figure(figsize=(14, 5))

    # Plot Losses
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, train_losses, label='Train Loss')
    plt.plot(epochs_range, val_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Loss')

    # Plot Accuracies
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, train_accuracies, label='Train Accuracy')
    plt.plot(epochs_range, val_accuracies, label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title('Training and Validation Accuracy')

    plt.tight_layout()
    plt.show()

plot_metrics(train_losses, val_losses, train_accuracies, val_accuracies)

Evaluation

In [ ]:
y_pred, y_true = [], []

def evaluate_model(model, test_loader):
    model.eval()
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs.to(device))
            _, predicted = torch.max(outputs, 1)
            y_pred.extend(predicted.cpu().numpy())
            y_true.extend(labels.cpu().numpy())

In [ ]:
evaluate_model(best_model, test_loader)

In [ ]:
m = confusion_matrix(y_true, y_pred)
cm_normalized = cm / cm.sum(axis=1)[:, np.newaxis]  # Normalize by row (true condition)

# Plot the normalized confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm_normalized, display_labels=CLASSES)
disp.plot(cmap=plt.cm.Blues, values_format=".0%")
plt.title('Normalized Confusion Matrix')
plt.show()

In [ ]:
print(classification_report(y_true, y_pred, target_names=CLASSES))